In [1]:
import csv
from pymongo import MongoClient
DB_CONNECTION = 'mongodb://localhost:27017'

In [2]:
HEADER_ROW = ["PS_TERM_ID","INSTRUCTOR_NAME","COURSE_UNIQUE_ID",
          "RESPONSE_UNIQUE_ID","QUESTION_ID","RESPONSE_TEXT",
          "RESPONSE_NUM","INSTRUCTOR_ENROLLMENTS","STUDENT_ENROLLMENTS",
          "STUDENT_RESPONDENTS","COURSE_RESPONSE_RATE","OPTOUTS",
          "OPTOUT_RATE","RESPONDENTS_COMPLETE","RESPONDENTS_COMPLETE_RATE",
          "COURSE_TITLE","PS_COURSE_ID","PS_CLASS_ID","PS_COMBINED_SECTION_ID",
          "CLASS_TYPE","CLASS_LEVEL","DEPT_CODE","SCHOOL_CODE","SCHOOL_NAME",
          "DEPT_NAME","CLASS_LOCATION","INSTRUCTION_MODE","QUESTION_TEXT",
          "ANSWER_SET_TEXT","ANSWER_SET_NUM"]

In [ ]:
with open('./res/') as csvfile:
    reader = csv.reader(csvfile,delimiter=',')
    client = MongoClient(DB_CONNECTION)
    for r in reader:
        entry = {key:r[index] for index,key in enumerate(HEADER_ROW)}
        client.course_evals.entries.insert_one(entry)